## Импорты и проч

In [15]:
# Импортируем необходимые библиотеки и функции
import os
import json
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf
import time

from baskerville import seqnn, gene as bgene
from borzoi_helpers import process_sequence, predict_tracks  # предполагается, что эти функции доступны

# Отключаем лишние предупреждения TensorFlow
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
!nvcc -V

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.14.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
%%bash

#Download model weights (data fold 3, 4 replicates)
for rep in f3c0,f0 f3c1,f1 f3c2,f2 f3c3,f3; do IFS=","; set -- $rep; 
  mkdir -p "saved_models/$1/train"
  local_model="saved_models/$1/train/model0_best.h5"
  if [ -f "$local_model" ]; then
    echo "$1 model already exists."
  else
    wget --progress=bar:force "https://storage.googleapis.com/seqnn-share/borzoi/$2/model0_best.h5" -O "$local_model"
  fi
done

#Download and uncompress annotation files
mkdir -p hg38/genes/gencode41
mkdir -p hg38/genes/polyadb

if [ -f hg38/genes/gencode41/gencode41_basic_nort.gtf ]; then
  echo "Gene annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_nort_protein.gtf ]; then
  echo "Gene annotation (no read-through, protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein.gtf ]; then
  echo "Gene annotation (protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_tss2.bed ]; then
  echo "TSS annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_tss2.bed.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_tss2.bed
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz ]; then
  echo "Splice site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.csv.gz -O hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.gff ]; then
  echo "Splice site annotation already exist."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.gff.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein_splice.gff
fi

if [ -f hg38/genes/polyadb/polyadb_human_v3.csv.gz ]; then
  echo "PolyA site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/polyadb_human_v3.csv.gz -O hg38/genes/polyadb/polyadb_human_v3.csv.gz
fi

#Download and index hg38 genome
mkdir -p hg38/assembly/ucsc

if [ -f hg38/assembly/ucsc/hg38.fa ]; then
  echo "Human genome FASTA already exists."
else
  wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > hg38/assembly/ucsc/hg38.fa
fi


f3c0 model already exists.
f3c1 model already exists.
f3c2 model already exists.
f3c3 model already exists.
Gene annotation already exists.
Gene annotation (no read-through, protein-coding) already exists.
lready exists.n (protein-coding) a
TSS annotation already exists.
Splice site annotation already exist.
tation already exist.
PolyA site annotation already exist.
Human genome FASTA already exists.


## Препроцессинг

Код для понимания какой индекс отвечает за какую клеточную линию

In [3]:
import pandas as pd

targets_file = 'targets_gtex.txt'
targets_df = pd.read_csv(targets_file, sep='\t', index_col=0)

# Чтобы иметь явный индекс внутри датафрейма для читаемости
targets_df['local_index'] = range(len(targets_df))

# Ткани, которые нас интересуют
tissue_list = [
    "kidney", 
    "liver",
    "adrenal",
    "pancreas",
    "lung"
]

print("Список каналов для нужных тканей:\n")

for tissue in tissue_list:
    # Поищем все строки, где в описании есть это слово/фраза (без учёта регистра)
    matches = targets_df[ targets_df['description'].str.lower().str.contains(tissue.lower()) ]
    
    if len(matches) == 0:
        print(f"Ткань '{tissue}' не найдена в targets_df['description'].")
    else:
        print(f"Ткань '{tissue}':")
        display(matches[['description','local_index']])
        print("-"*50)


Список каналов для нужных тканей:

Ткань 'kidney':


,description,local_index
7560,RNA:kidney,38
7561,RNA:kidney,39
7562,RNA:kidney,40


--------------------------------------------------
Ткань 'liver':


,description,local_index
7563,RNA:liver,41
7564,RNA:liver,42
7565,RNA:liver,43


--------------------------------------------------
Ткань 'adrenal':


...

--------------------------------------------------
Ткань 'pancreas':


,description,local_index
7577,RNA:pancreas,55
7578,RNA:pancreas,56
7579,RNA:pancreas,57


--------------------------------------------------
Ткань 'lung':


,description,local_index
7566,RNA:lung,44
7567,RNA:lung,45
7568,RNA:lung,46


--------------------------------------------------


In [64]:
targets_df

,identifier,file,clip,clip_soft,scale,sum_stat,strand_pair,description
7522,GTEX-132QS-2526-SM-62LFJ.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7522,RNA:adipose_tissue
7523,GTEX-1GMR3-0826-SM-9WYT4.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7523,RNA:adipose_tissue
7524,GTEX-1HSEH-0226-SM-ACKVV.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7524,RNA:adipose_tissue
7525,GTEX-11GSP-0326-SM-5A5KW.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7525,RNA:adrenal_gland
7526,GTEX-13PVR-0226-SM-5RQJI.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7526,RNA:adrenal_gland
...,...,...,...,...,...,...,...,...
7606,GTEX-13FTX-1026-SM-5J2O5.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7606,RNA:uterus
7607,GTEX-1MA7W-1526-SM-DHXKS.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7607,RNA:uterus
7608,GTEX-11EMC-1926-SM-5A5JU.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7608,RNA:vagina
7609,GTEX-12WSB-2426-SM-5EGJC.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7609,RNA:vagina


In [4]:
kidney_ix =  39  
liver_ix =   42  # "left lobe of liver tissue"
adrenal_ix = 4
pancreas_ix =56
lung_ix =    45

# Словарь: название ткани -> индекс канала модели
channels_dict = {
    "ENCFF123KIW_kidney"                 : kidney_ix,
    "ENCFF784MDF_left_lobe_of_liver"     : liver_ix,
    "ENCFF236XOK_adrenal_gland"          : adrenal_ix,
    "ENCFF781TTC_pancreas"               : pancreas_ix,
    "ENCFF242BWW_lung"                   : lung_ix
}

In [5]:
import pandas as pd
import gzip

# Загрузим файл .bed.gz с последовательностями
bed_file_path = '../data/sequences_human.bed.gz'

# Открываем и загружаем данные из .bed файла
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, names=["chrom", "start", "end", "fold"])

# Фильтрация данных по фолду
folds_to_process = ['fold3']  # Можно изменить на нужные фолды
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Сгруппируем по хромосомам
grouped = filtered_bed_data.groupby("chrom")

print(len(filtered_bed_data))
# Посмотрим первые несколько строк
filtered_bed_data.head()

6888


,chrom,start,end,fold
20329,chr5,41626145,41822753,fold3
20330,chr11,40389266,40585874,fold3
20331,chr11,32570759,32767367,fold3
20332,chr6,164265822,164462430,fold3
20333,chr5,13736747,13933355,fold3


In [6]:
from tqdm import tqdm

def merge_intervals(intervals):
    """
    Принимает список интервалов вида [(start, end), ...] и возвращает список объединённых интервалов.
    """
    # Сортируем интервалы по start
    intervals.sort(key=lambda x: x[0])
    merged = []
    for start, end in intervals:
        if not merged or start > merged[-1][1]:
            merged.append([start, end])
        else:
            merged[-1][1] = max(merged[-1][1], end)
    return merged

print("Количество интервалов до мерджа:", len(filtered_bed_data))

# Объединяем интервалы по каждой хромосоме с прогресс-баром.
merged_intervals_list = []
# Количество групп = число уникальных хромосом в filtered_bed_data
unique_chroms = filtered_bed_data['chrom'].nunique()
for chrom, group in tqdm(filtered_bed_data.groupby("chrom"), desc="Мержим интервалы", total=unique_chroms):
    # Получаем список интервалов для данной хромосомы
    intervals = group[['start', 'end']].values.tolist()
    merged = merge_intervals(intervals)
    # Если все интервалы имеют одинаковый fold, то просто берем первый
    fold_val = group['fold'].iloc[0]
    for start, end in merged:
        merged_intervals_list.append({'chrom': chrom, 'start': start, 'end': end, 'fold': fold_val})

# Перезаписываем filtered_bed_data объединёнными интервалами
filtered_bed_data = pd.DataFrame(merged_intervals_list)

# Выведем результат для проверки
print("Количество интервалов после мерджа:", len(filtered_bed_data))
filtered_bed_data.head()


Количество интервалов до мерджа: 6888


Мержим интервалы: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 965.55it/s]

Количество интервалов после мерджа: 46


,chrom,start,end,fold
0,chr1,143479625,143823752,fold3
1,chr1,227321006,228550247,fold3
2,chr11,25440674,48748592,fold3
3,chr11,48896195,49191149,fold3
4,chr11,54525074,54820028,fold3


## Проверка данных

Убедимся что все записи влезут во входящее окно модели

In [6]:
# Находим максимальную длину последовательности
max_sequence_length = 0

for index, row in filtered_bed_data.iterrows():
    start = row['start']
    end = row['end']
    
    # Вычисляем длину текущей последовательности
    sequence_length = end - start
    
    # Обновляем максимальную длину, если текущая больше
    if sequence_length > max_sequence_length:
        max_sequence_length = sequence_length

print(f"Максимальная длина последовательности: {max_sequence_length} нуклеотидов")


Максимальная длина последовательности: 67957002 нуклеотидов


Проверка на пересечения

In [7]:
import pandas as pd
from tqdm import tqdm

overlap_found = False

# Получаем список уникальных хромосом
chromosomes = filtered_bed_data['chrom'].unique()

# Используем tqdm для отображения прогресса
for chrom in tqdm(chromosomes, desc="Checking for overlaps"):
    # Фильтруем по текущей хромосоме
    group = filtered_bed_data[filtered_bed_data['chrom'] == chrom]
    
    # Сортируем записи по старту
    sorted_group = group.sort_values('start')
    
    # Инициализируем предыдущую запись
    prev_row = None
    
    # Проходим по отсортированным записям
    for idx, row in sorted_group.iterrows():
        if prev_row is not None:
            # Проверяем пересечение
            if row['start'] < prev_row['end']:
                print(f"Пересечение на {chrom}: {prev_row[['start', 'end']].to_dict()} и {row[['start', 'end']].to_dict()}")
                overlap_found = True
        prev_row = row

if not overlap_found:
    print("Пересечений не обнаружено в filtered_bed_data.")


Checking for overlaps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1105.77it/s]

Пересечений не обнаружено в filtered_bed_data.


In [8]:
import pandas as pd
import gzip

# Загрузим данные из bed-файла
bed_file_path = '../data/sequences_human.bed.gz'
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, 
                           names=["chrom", "start", "end", "fold"])

# Фильтруем данные по fold3 и fold4
#folds_to_process = ['fold3', 'fold4']
folds_to_process = ['fold3']
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Выведем список уникальных хромосом
unique_chroms = filtered_bed_data['chrom'].unique()
print("Уникальные хромосомы в выборке:")
print(unique_chroms)
print(f"Количество уникальных хромосом: {len(unique_chroms)}")

# Для каждой хромосомы находим минимальный start и максимальный end
chrom_stats = filtered_bed_data.groupby("chrom").agg(min_start=('start', 'min'),
                                                      max_end=('end', 'max'))
print("\nМинимальный start и максимальный end по хромосомам:")
print(chrom_stats)

if len(unique_chroms) > 1:
    print("\nВыборка содержит более одной хромосомы.")
else:
    print("\nВыборка содержит только одну хромосому.")


Уникальные хромосомы в выборке:
['chr5' 'chr11' 'chr6' 'chr9' 'chr13' 'chr2' 'chr8' 'chr7' 'chr17' 'chrX'
 'chr18' 'chr12' 'chr15' 'chr19' 'chr1' 'chr16' 'chr20']
Количество уникальных хромосом: 17

Минимальный start и максимальный end по хромосомам:
       min_start    max_end
chrom                      
chr1   143479625  228550247
chr11   25440674   56541083
chr12   32992234   34368994
chr13   40653298  102135774
chr15   20168638   34675550
chr16   33491584   33884884
chr17   81799133   83225066
chr18   13485786   15206757
chr19    3108622    9156817
chr2    68747521  129505661
chr20   30186694   30383302
chr5     8770274  103148800
chr6    99603327  170690035
chr7       10000   74778699
chr8     8158857   55824055
chr9    80588528  138217638
chrX     3230043   58109050

Выборка содержит более одной хромосомы.


## Модель

In [7]:
from baskerville import seqnn

params_file = 'params_pred.json'
with open(params_file) as f:
    params = json.load(f)

params_model = params['model']
params_train = params['train']

model_file = 'saved_models/f3c0/train/model0_best.h5'
rc = False  # Мы будем сами RC предсказывать, если нужно
n_reps = 1

targets_file = 'targets_gtex.txt'
targets_df = pd.read_csv(targets_file, sep='\t', index_col=0)
target_index = targets_df.index

# Загружаем модель
seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(model_file, 0)
seqnn_model.build_slice(target_index)
seqnn_model.build_ensemble(rc, [0])

models = [seqnn_model]
print("Модель загружена и готова.")

# Вытаскиваем stride, crops и т.д.
stride = seqnn_model.model_strides[0]
crop   = seqnn_model.target_crops[0]     # напр. может быть ~768
tlen   = seqnn_model.target_lengths[0]   # напр. ~6400-7000, зависит от версии

print("Параметры модели:")
print(f"  stride={stride}, crop={crop}, target_length={tlen}")


2025-02-18 23:41:28.503092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23390 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:01.0, compute capability: 7.0
2025-02-18 23:41:28.503827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30965 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:02.0, compute capability: 7.0


Модель загружена и готова.
Параметры модели:
  stride=32, crop=16, target_length=16352


In [8]:
fasta_path = "hg38/assembly/ucsc/hg38.fa"
fasta_index = pysam.Fastafile(fasta_path)

chr_sizes = {
    chrom: fasta_index.get_reference_length(chrom)
    for chrom in fasta_index.references
}

print("Пример ключей chr_sizes:", list(chr_sizes.keys())[:5])


Пример ключей chr_sizes: ['chr1', 'chr10', 'chr11', 'chr11_KI270721v1_random', 'chr12']


# Инференс

In [9]:
center_len = 196_608
context_len = 163_840

all_windows = []
for _, row in filtered_bed_data.iterrows():
    chrom = row['chrom']
    start_merged = row['start']
    end_merged   = row['end']
    curr_start = start_merged
    while curr_start < end_merged:
        curr_end = curr_start + center_len
        if curr_end > end_merged:
            curr_end = end_merged
        all_windows.append((chrom, curr_start, curr_end))
        curr_start = curr_end

print(f"Всего сформировано {len(all_windows)} окон.")


Всего сформировано 1785 окон.


In [10]:
# --- ЯЧЕЙКА 8: Предсоздание bedGraph файлов ---

output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

folds_str = "_".join(folds_to_process)
chains = ["st+", "st-"]

for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        with open(file_path, "w") as f:
            f.write(f"track type=bedGraph name=\"{bed_name} {chain}\"\n")

print("Предсозданы файлы .bedGraph:")
for bed_name in channels_dict.keys():
    for chain in chains:
        fp = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        print(fp)


Предсозданы файлы .bedGraph:
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF242BWW_lung.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF242BWW_lung.bedGraph


In [18]:
def reverse_complement_onehot(seq_onehot):
    """Обращаем (L,4) → (L,4) реверс-комплемент."""
    rev = np.flip(seq_onehot, axis=0)
    revcomp = rev[:, [3,2,1,0]]
    return revcomp


def process_window(chrom, center_start, center_end, context_len, models):
    """
    Выполняем:
      1) Берём вход с запасом context_len.
      2) Предсказываем +цепь (y_plus).
      3) Предсказываем -цепь (y_minus), через reverse complement.
      4) Вырезаем из выходных массивов только центральный диапазон,
         учитывая crop.
      5) Возвращаем (center_pred_plus, center_pred_minus), 
         размер [ (center_end-center_start)//stride , num_targets ].
         Либо (None, None), если не влезает.
    """

    input_start = max(0, center_start - context_len)
    input_end   = min(chr_sizes[chrom], center_end + context_len)

    # Получаем one-hot
    seq_plus = process_sequence(fasta_index, chrom, input_start, input_end)
    if seq_plus is None or seq_plus.shape[0] == 0:
        return None, None

    # Предсказание для + цепи
    y_plus = predict_tracks(models, seq_plus)  # [1, L_out, 1, C] или [L_out, C]
    #y_plus = np.squeeze(y_plus, axis=(0,2))  # теперь [L_out, C]
    y_plus = y_plus[0, :, 0, :]


    # Предсказание для - цепи
    seq_minus = reverse_complement_onehot(seq_plus)
    y_minus = predict_tracks(models, seq_minus)
    #y_minus = np.squeeze(y_minus, axis=(0,2))
    y_minus = y_minus[0, :, 0, :]


    # Определим, сколько «выходных позиций» даёт модель на вход длины L_in.
    # Но проще полагаться на shape y_plus:
    L_out = y_plus.shape[0]

    # Посчитаем, на сколько нуклеотидов сдвинут реальный "0-й" выход
    # относительно входа. По умолчанию borzoi обрезает crop * stride с краёв:
    offset_nt = stride * crop  # к примеру, 768*128 = 98304

    # Где начинается «полезный выход» на линейке входа:
    out_start_on_input = offset_nt  
    # Где заканчивается «полезный выход» (чтобы не вылезти за L_out)
    out_end_on_input   = offset_nt + stride*(L_out - 2*crop)

    # Теперь нужно вырезать именно диапазон [center_start : center_end] 
    # (в координатах input), но исключив краевые эффекты.
    # Местоположение center_start в "координатах входа":
    center_start_in_input = center_start - input_start
    center_end_in_input   = center_end   - input_start

    # Пересекаем это всё с [out_start_on_input, out_end_on_input]
    slice_start = max(center_start_in_input, out_start_on_input)
    slice_end   = min(center_end_in_input, out_end_on_input)

    # Проверим, есть ли вообще что вырезать:
    if slice_end <= slice_start:
        return None, None

    # Индексы в выходном тензоре y_plus (по оси L_out):
    out_slice_start = int((slice_start - out_start_on_input)//stride)
    out_slice_end   = int(np.ceil((slice_end - out_start_on_input)/stride))

    # Если выходим за границы массива:
    if out_slice_start < 0:
        out_slice_start = 0
    if out_slice_end > (L_out - 2*crop):
        out_slice_end = L_out - 2*crop
    if out_slice_end <= out_slice_start:
        return None, None

    # Наконец берем нужный кусок из y_plus / y_minus
    # Но учтём, что «физически» в y_plus массиве полный размер L_out,
    # а обрезка от crop до (L_out - crop) обычно и есть «рабочая» зона.
    # Проще сместить всё на crop:
    out_slice_start_full = crop + out_slice_start
    out_slice_end_full   = crop + out_slice_end

    center_pred_plus = y_plus[out_slice_start_full : out_slice_end_full, :]
    center_pred_minus = y_minus[out_slice_start_full : out_slice_end_full, :]

    # Возвращаем
    return center_pred_plus, center_pred_minus


def run_inference(all_windows, models):
    """
    Для каждого (chrom, cstart, cend) → предсказываем 
    center_pred_plus, center_pred_minus → записываем bedGraph.
    """
    start_time = time.time()
    print(f"Начинаем инференс по {len(all_windows)} окнам.")

    for (chrom, cstart, cend) in tqdm(all_windows, desc="Inference windows"):
        center_pred_plus, center_pred_minus = process_window(chrom, cstart, cend, context_len, models)
        if center_pred_plus is None or center_pred_minus is None:
            continue

        # Запись в bedGraph
        length_out = center_pred_plus.shape[0]  # кол-во позиций после обрезки
        for bed_name, channel_ix in channels_dict.items():
            # + цепь
            if channel_ix < center_pred_plus.shape[1]:
                vals_plus = center_pred_plus[:, channel_ix]
                file_path_plus = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_st+_{bed_name}.bedGraph")
                
                with open(file_path_plus, "a") as fp:
                    for i, val in enumerate(vals_plus):
                        # Каждая позиция i в center_pred_plus
                        # соответствует real_start = cstart + (i*stride) + "смещение" ...
                        # Но мы уже в process_window аккуратно вырезали «правильную» часть.
                        # Чтобы просто положить их на [cstart:cend], будем идти равномерно:
                        pos_start = cstart + i*stride
                        pos_end   = pos_start + stride
                        if pos_end > cend:
                            pos_end = cend
                        if pos_end <= pos_start:
                            break
                        fp.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

            # - цепь
            if channel_ix < center_pred_minus.shape[1]:
                vals_minus = center_pred_minus[:, channel_ix]
                file_path_minus = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_st-_{bed_name}.bedGraph")
                
                with open(file_path_minus, "a") as fm:
                    for i, val in enumerate(vals_minus):
                        pos_start = cstart + i*stride
                        pos_end   = pos_start + stride
                        if pos_end > cend:
                            pos_end = cend
                        if pos_end <= pos_start:
                            break
                        fm.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

    elapsed = time.time() - start_time
    print(f"Инференс завершён за {elapsed:.2f} секунд.")


In [ ]:
# --- ЯЧЕЙКА 9: Запуск инференса ---

print("Start inference...")

run_inference(all_windows, models)

print("Done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.")


Start inference...
Начинаем инференс по 1785 окнам.


Inference windows:   2%|██▎                                                                                                                           | 32/1785 [01:02<57:25,  1.97s/it]